# droitGPT
### AI Assistant specialized in French legal codes

# The language model
### Qwen-1_8B-Chat-Int4

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("backend/tokenizer/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("backend/llm/", device_map="auto", trust_remote_code=True).eval()

user_q = ["Salut!", "Quelle est la capitale du Pérou?", "Qui est le président du Pérou?"]

history_in = None
for query in user_q:
    response, history_out = model.chat(tokenizer, query, history=history_in)
    print("User: ", query)
    print("Assistant: ", response)
    print("Chat history: ", history_out)
    history_in = history_out
    print()

User:  Salut!
Assistant:  Bonjour! Comment puis-je vous aider aujourd'hui?
Chat history:  [('Salut!', "Bonjour! Comment puis-je vous aider aujourd'hui?")]

User:  Quelle est la capitale du Pérou?
Assistant:  La capitale du Perou est Abénaï.
Chat history:  [('Salut!', "Bonjour! Comment puis-je vous aider aujourd'hui?"), ('Quelle est la capitale du Pérou?', 'La capitale du Perou est Abénaï.')]

User:  Qui est le président du Pérou?
Assistant:  Le président du Perou actuel est Jamila Mahmoudine Boulidi.
Chat history:  [('Salut!', "Bonjour! Comment puis-je vous aider aujourd'hui?"), ('Quelle est la capitale du Pérou?', 'La capitale du Perou est Abénaï.'), ('Qui est le président du Pérou?', 'Le président du Perou actuel est Jamila Mahmoudine Boulidi.')]



# The Embeddings Model 
### Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

/home/adrian/Desktop/MinesParistech2023-2024/NLP/droitGPT/backend/venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Vector Database
### FAISS

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document


docs = [Document(page_content="Le suffrage est direct et universel."),
        Document(page_content="Sont électeurs les Françaises et Français âgés de dix-huit ans accomplis, jouissant de leurs droits civils et politiques et n'étant dans aucun cas d'incapacité prévu par la loi."),
        Document(page_content="Ne doivent pas être inscrits sur la liste électorale, pendant le délai fixé par le jugement, ceux auxquels les tribunaux ont interdit le droit de vote et d'élection, par application des lois qui autorisent cette interdiction."),
        ]
vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont un peu similaires au requête (modèle Embeddings n'est pas très grand)

In [10]:
def print_context(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [11]:
query = "Qui sont les electeurs?"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
Ne doivent pas être inscrits sur la liste électorale, pendant le délai fixé par le jugement, ceux auxquels les tribunaux ont interdit le droit de vote et d'élection, par application des lois qui autorisent cette interdiction.

Top 2
Sont électeurs les Françaises et Français âgés de dix-huit ans accomplis, jouissant de leurs droits civils et politiques et n'étant dans aucun cas d'incapacité prévu par la loi.

Top 3
Le suffrage est direct et universel.



# Corpus

In [14]:
import os 
import re

clean_data_folder = "new_clean_data"
data_folder = "backend/data"
files_for_indexing = ["electoral.md"]

# TODO
# remove title and date
# remove html content

# divide by markdown title and get title name
# do this recursive until there is no title
# create file name based on titles 
# save file name

def clean_data():
    if not os.path.exists(clean_data_folder):
        os.makedirs(clean_data_folder)

    for file in os.listdir(data_folder):
        if file in files_for_indexing:
            file_path = data_folder + "/" + file
            
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()

                pattern1 = re.compile(r"---.*?---", re.DOTALL)  # remove title and date
                text = re.sub(pattern1, "", text)

                pattern3 = re.compile(
                    r"<div.*</div>"
                )  # remove html content (simple approach, usually tables)
                text = re.sub(pattern3, "", text)

                pattern4 = re.compile(r"([:;])\n+")  # form paragraphs
                text = re.sub(pattern4, lambda x: x.group(1) + " ", text)

                articles = re.findall(r"\*\*(Art\. [^\*]+)\*\*\n([^\*]+)", text.strip())
                for article_name, text in articles:
                    if "tableau" not in article_name:
                        clean_file_path = clean_data_folder + "/" + article_name + "_" + file
                        with open(clean_file_path, "w", encoding="utf-8") as f:
                            f.write(text.strip())
clean_data()

In [2]:
import markdown_it

def parse_markdown_to_dict(markdown_content):
    md = markdown_it.MarkdownIt()

    tokens = md.parse(markdown_content, {})
    
    result_dict = {}
    current_key = None
    current_value = []

    for token in tokens:
        if token.type == 'heading_open':
            current_key = token['tag']
            current_value = []
        elif token.type == 'inline':
            current_value.append(token['content'])
        elif token.type == 'heading_close':
            result_dict[current_key] = ''.join(current_value).strip()
    
    return result_dict

In [3]:
import markdown_it

def parse_markdown_to_nested_dict(markdown_content):
    md = markdown_it.MarkdownIt()

    tokens = md.parse(markdown_content, {})

    def process_tokens(tokens):
        result_dict = {}
        current_key = None
        current_value = []

        for token in tokens:
            if token.type == 'heading_open':
                heading_level = int(token.tag[1])
                current_key = token.content
                current_value = process_tokens(tokens)
                result_dict[heading_level] = {current_key: current_value}
            elif token.type == "inline":
                current_value.append(token.content)
            elif token.type == 'heading_close':
                return ''.join(current_value).strip()

        return result_dict

    return process_tokens(tokens)

In [4]:
file = "electoral.md"
file_path = data_folder + "/" + file
clean_file_path = clean_data_folder + "/clean_" + file

with open(file_path, 'r', encoding='utf-8') as markdown_file:
        markdown_content = markdown_file.read()
        parsed_dict = parse_markdown_to_nested_dict(markdown_content)
        print(parsed_dict)

RecursionError: maximum recursion depth exceeded while calling a Python object